In [12]:
from gurobipy import *
import pandas as pd

In [13]:
# df1_1 = pd.read_excel("OR110-1_case01.xlsx", "Instance 1")
# df1_2 = pd.read_excel("OR110-1_case01.xlsx", "Instance 2")
df1_1 = pd.read_excel("OR110-1_case01.xlsx", "Instance 3")

# display(df1_1)

df1_1 = df1_1.rename(columns={'Process Type': 'type_P1',
                              'Unnamed: 2': 'type_P2',
                              'Unnamed: 3': 'type_P3', 
                              'Unnamed: 4': 'type_P4', 
                              'Unnamed: 5': 'type_P5',
                              'Processing Time': 'time_P1', 
                              'Unnamed: 8': 'time_P2',
                              'Unnamed: 9': 'time_P3',
                              'Unnamed: 10': 'time_P4',
                              'Unnamed: 11': 'time_P5',
                             })
df1_1 = df1_1.drop(0)

df1_1 = df1_1.fillna(0)

display(df1_1)
# display(df1_2)
# display(df1_3)

,Job ID,type_P1,type_P2,type_P3,type_P4,type_P5,Splitting Timing,time_P1,time_P2,time_P3,time_P4,time_P5,Due Time
1,1.0,Boiling,Baking,Smoking,0,0,2.0,2.0,1.0,1.5,0.0,0.0,12:30:00
2,2.0,Boiling,Baking,Boiling,0,0,2.0,1.6,1.4,2.0,0.0,0.0,12:30:00
3,3.0,Boiling,Baking,Boiling,Baking,Boiling,1.0,3.0,0.8,1.1,0.3,1.2,12:30:00
4,4.0,Smoking,Boiling,Baking,Smoking,Boiling,4.0,0.3,1.2,1.0,0.5,0.6,12:30:00
5,5.0,Boiling,Baking,Boiling,0,0,1.0,1.2,0.7,1.8,0.0,0.0,12:30:00
6,6.0,Boiling,0,0,0,0,0.0,3.3,0.0,0.0,0.0,0.0,12:30:00
7,7.0,Baking,Smoking,0,0,0,1.0,1.7,1.5,0.0,0.0,0.0,17:30:00
8,8.0,Baking,0,0,0,0,0.0,1.3,0.0,0.0,0.0,0.0,17:30:00
9,9.0,Boiling,Baking,Boiling,0,0,2.0,0.8,1.0,2.2,0.0,0.0,17:30:00
10,10.0,Baking,Smoking,Boiling,Baking,Smoking,3.0,0.6,1.3,0.7,0.2,1.1,17:30:00


In [14]:
n = len(df1_1["Job ID"])
J = range(n)
I = range(5)
L = range(2)

# K = [(j, k) for j in J for k in J if j < k]
JL = [(j, l) for j in J for l in L]
X = [(i, j) for i in I for j in JL]
H = [(j, k) for j in JL for k in JL if j != k]

Pn = 5

P = []
for i in J:
    P.append([])
    st = int(df1_1['Splitting Timing'][i+1])
    tmpP = 0
    for j in range(1, st+1):
        tmpStr = "time_P" + str(j)
        tmpP += df1_1[tmpStr][i+1]
    P[i].append(tmpP)
    idx = st+1
    tmpP = 0
    tmpStr = "time_P" + str(idx)
    while idx<=Pn and df1_1[tmpStr][i+1] != 0:
        tmpP += df1_1[tmpStr][i+1]
        idx += 1
        tmpStr = "time_P" + str(idx)
    P[i].append(tmpP)
    
display(P)

M1 = 0
for i in P:
    for j in i:
        M1 += j
        
print(M1)

[[3.0, 1.5],
 [3.0, 2.0],
 [3.0, 3.4000000000000004],
 [3.0, 0.6],
 [1.2, 2.5],
 [0, 3.3],
 [1.7, 1.5],
 [0, 1.3],
 [1.8, 2.2],
 [2.5999999999999996, 1.3]]

38.9


In [15]:
import datetime

start = datetime.time(7, 30, 0)
start = datetime.datetime.combine(datetime.date.today(), start)

D = []

for i in df1_1["Due Time"]:
    a = datetime.datetime.combine(datetime.date.today(), i) - start
    D.append(int(a.seconds / 3600))
    #due.append(datetime.combine(date.today(), i) - datetime.combine(date.today(), start))

print(D)

[5, 5, 5, 5, 5, 5, 10, 10, 10, 10]


In [16]:
B = []
for i in J:
    B.append([])
    st = int(df1_1['Splitting Timing'][i+1])
    if st == 1:
        tmpStr = "type_P" + str(st)
        if df1_1[tmpStr][i+1] == 'Boiling':
            B[i].append(1)
        else:
            B[i].append(0)
    else:
        B[i].append(0)
    idx = st+1
    cnt = 0
    tmpStr = "type_P" + str(idx)
    while idx<=Pn and df1_1[tmpStr][i+1] != 0:
        cnt += 1
        idx += 1
        tmpStr = "type_P" + str(idx)
    if cnt == 1:
        if df1_1["type_P" + str(idx-1)][i+1] == 'Boiling':
            B[i].append(1)
        else:
            B[i].append(0)
    else:
        B[i].append(0)
    
display(B)

[[0, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 0],
 [0, 0],
 [0, 1],
 [0, 0]]

In [17]:
eg2 = Model("problem3")    # build a new model

x = eg2.addVars(I, JL, lb = 0, vtype = GRB.BINARY, name = "x")
z = eg2.addVars(H, lb = 0, vtype = GRB.BINARY, name = "z")
t = eg2.addVars(J, lb = 0, vtype = GRB.BINARY, name = "t")
f = eg2.addVars(JL, lb = 0, vtype = GRB.CONTINUOUS, name = "f")

In [18]:
# setting the objective function 
eg2.setObjective(
    quicksum(t[j] for j in J)
    , GRB.MINIMIZE) 


# add constraints and name them

for j, l in JL:
    eg2.addConstr(x[1, j, l] <= B[j][l], name = f"B_{j}_{l}")

for j in J:
    eg2.addConstr(f[j, 0] + P[j][1] <= f[j, 1], name = f"job_{j+1} part1 before part2")

for j in J:
    eg2.addConstr((f[j, 1] - D[j] <= M1 * t[j]), name = f"define t_{j+1}")
    
for j, l in JL:
    eg2.addConstr(quicksum(x[i, j, l] for i in I) == 1, name = f"job {j+1} in one machine")
    
# for j, k in H:
#     eg2.addConstr(z[j, k] + z[k, j] <= 1, name = "z")

for j, k in H:
    for i in I:
        eg2.addConstr(z[j, k] + z[k, j] + 1 >= x[i, j[0], j[1]] + x[i, k[0], k[1]], name = f"define z_{j[0]+1}_{k[0]+1}")
#         eg2.addConstr(x[i, j] + x[i, k] <= 2, name = f"define x")
    
for j, k in H:
    eg2.addConstr(f[j] + P[k[0]][k[1]] - f[k] <= M1 * (1 - z[j, k]), name = f"define f_{k[0]+1}")
#     eg2.addConstr(f[k] + P[j] - f[j] <= M1 * (1 - z[k, j]), name = f"define f_{j[0]+1}")

for j in JL:
    eg2.addConstr(f[j] >= P[j[0]][j[1]], name = f"f_{j[0]+1} > P_{j[0]+1}")
    
eg2.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 2360 rows, 510 columns and 8920 nonzeros
Model fingerprint: 0x1faa8ec1
Variable types: 20 continuous, 490 integer (490 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e-01, 4e+01]
Presolve removed 1216 rows and 35 columns
Presolve time: 0.01s
Presolved: 1144 rows, 475 columns, 4164 nonzeros
Variable types: 20 continuous, 455 integer (455 binary)
Found heuristic solution: objective 6.0000000

Root relaxation: objective 1.000000e+00, 42 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.00000    0    2    6.00000    1.00000  83.3%     -    0s
H    0     0                 

In [19]:
print("Result:")

# objecitve value
print("z* =", eg2.objVal)
print("")

# t
for j in J:
    print(f[j, 0].varName, '=', round(f[j, 0].x, 2), ",", f[j, 1].varName, '=', round(f[j, 1].x, 2), ', Due time =', D[j], ',', t[j].varName, '=', t[j].x)
print("")
    
# x
print("x:")
print("j \ i", end="")
print("\tMachine2 Machine3 Machine4 Machine5")
for j, l in JL:
    # mark which product is printed now
    if len(str("Job" + str(j+1) + "-" + str(l+1))) < 7:
        print("Job" + str(j+1) + "-" + str(l+1), "\t", end="")
    else:
        print("Job" + str(j+1) + "-" + str(l+1), "", end="")
    for i in I:
        # print values of each kind of product
        if len(str(x[i, j, l].x)) < 7:
            print(x[i, j, l].x, "\t", end="")
        else:
            print(x[i, j, l].x, "", end="")
    print("")    # use for change line
print("")

# z
print("z:")
print("j \ k", end="\t")
for j, l in JL:
    print("J" + str(j+1) + "-" + str(l+1), " ", end="")
print("")    # use for change line
for j, l in JL:
    # mark which product is printed now
    if len(str("Job" + str(j+1) + "-" + str(l+1))) < 7:
        print("Job" + str(j+1) + "-" + str(l+1), "\t", end="")
    else:
        print("Job" + str(j+1) + "-" + str(l+1), "", end="")
    for k, m in JL:
        # print values of each kind of product
        if (j, l) != (k, m):
            if len(str(z[(j, l), (k, m)].x)) < 7:
                print(z[(j, l), (k, m)].x, "  ", end="")
            else:
                print(z[(j, l), (k, m)].x, "", end="")
        else:
            print("       ", end="")
    print("")    # use for change line
print("")

Result:
z* = 1.0

f[0,0] = 3.0 , f[0,1] = 4.5 , Due time = 5 , t[0] = 0.0
f[1,0] = 3.0 , f[1,1] = 5.0 , Due time = 5 , t[1] = 0.0
f[2,0] = 7.59 , f[2,1] = 16.4 , Due time = 5 , t[2] = 1.0
f[3,0] = 3.0 , f[3,1] = 3.6 , Due time = 5 , t[3] = 0.0
f[4,0] = 1.2 , f[4,1] = 3.8 , Due time = 5 , t[4] = 0.0
f[5,0] = 0.0 , f[5,1] = 4.5 , Due time = 5 , t[5] = 0.0
f[6,0] = 7.3 , f[6,1] = 8.8 , Due time = 10 , t[6] = 0.0
f[7,0] = 0.0 , f[7,1] = 1.3 , Due time = 10 , t[7] = 0.0
f[8,0] = 5.6 , f[8,1] = 9.3 , Due time = 10 , t[8] = 0.0
f[9,0] = 7.1 , f[9,1] = 8.4 , Due time = 10 , t[9] = 0.0

x:
j \ i	Machine2 Machine3 Machine4 Machine5
Job1-1 	1.0 	0.0 	0.0 	0.0 	0.0 	
Job1-2 	1.0 	0.0 	0.0 	0.0 	0.0 	
Job2-1 	0.0 	0.0 	1.0 	0.0 	0.0 	
Job2-2 	0.0 	0.0 	1.0 	0.0 	0.0 	
Job3-1 	0.0 	1.0 	0.0 	0.0 	0.0 	
Job3-2 	1.0 	0.0 	0.0 	0.0 	0.0 	
Job4-1 	0.0 	0.0 	0.0 	1.0 	0.0 	
Job4-2 	0.0 	0.0 	0.0 	1.0 	0.0 	
Job5-1 	0.0 	1.0 	0.0 	0.0 	0.0 	
Job5-2 	0.0 	0.0 	0.0 	0.0 	1.0 	
Job6-1 	0.0 	0.0 	0.0 	0.0 	1.

In [20]:
eg3 = Model("problem2-2")    # build a new model

x3 = eg3.addVars(I, JL, lb = 0, vtype = GRB.BINARY, name = "x")
z3 = eg3.addVars(H, lb = 0, vtype = GRB.BINARY, name = "z")
t3 = eg3.addVars(J, lb = 0, vtype = GRB.BINARY, name = "t")
f3 = eg3.addVars(JL, lb = 0, vtype = GRB.CONTINUOUS, name = "f")

w3 = eg3.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "w")

In [21]:
# setting the objective function 
eg3.setObjective(
    w3
    , GRB.MINIMIZE) 


# add constraints and name them

for j in J:
    eg3.addConstr(w3 >= f3[j, 1], name="w")

eg3.addConstr(quicksum(t3[j] for j in J) <= eg2.objVal, name = "limit tardy jobs")

for j, l in JL:
    eg3.addConstr(x3[1, j, l] <= B[j][l], name = f"B_{j}_{l}")

for j in J:
    eg3.addConstr(f3[j, 0] + P[j][1] <= f3[j, 1], name = f"job_{j+1} part1 before part2")

for j in J:
    eg3.addConstr((f3[j, 1] - D[j] <= M1 * t3[j]), name = f"define t_{j+1}")
    
for j, l in JL:
    eg3.addConstr(quicksum(x3[i, j, l] for i in I) == 1, name = f"job {j+1} in one machine")
    
# for j, k in H:
#     eg2.addConstr(z[j, k] + z[k, j] <= 1, name = "z")

for j, k in H:
    for i in I:
        eg3.addConstr(z3[j, k] + z3[k, j] + 1 >= x3[i, j[0], j[1]] + x3[i, k[0], k[1]], name = f"define z_{j[0]+1}_{k[0]+1}")
#         eg3.addConstr(x[i, j] + x[i, k] <= 2, name = f"define x")
    
for j, k in H:
    eg3.addConstr(f3[j] + P[k[0]][k[1]] - f3[k] <= M1 * (1 - z3[j, k]), name = f"define f_{k[0]+1}")
#     eg3.addConstr(f[k] + P[j] - f[j] <= M1 * (1 - z[k, j]), name = f"define f_{j[0]+1}")

for j in JL:
    eg3.addConstr(f3[j] >= P[j[0]][j[1]], name = f"f_{j[0]+1} > P_{j[0]+1}")
    
eg3.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 2371 rows, 511 columns and 8950 nonzeros
Model fingerprint: 0x619e1d1a
Variable types: 21 continuous, 490 integer (490 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e-01, 4e+01]
Presolve removed 1492 rows and 182 columns
Presolve time: 0.01s
Presolved: 879 rows, 329 columns, 2938 nonzeros
Variable types: 19 continuous, 310 integer (310 binary)

Root relaxation: objective 6.400000e+00, 154 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.40000    0   18          -    6.40000      -     -    0s
H    0     0                      11.3000000    6.40000  43.4%     -    0s

In [22]:
print("Result:")

# objecitve value
print("z* =", eg3.objVal)
print("")

# t
for j in J:
    print(f3[j, 0].varName, '=', round(f3[j, 0].x, 2), f3[j, 1].varName, '=', round(f3[j, 1].x, 2), ', Due time =', D[j], ',', t3[j].varName, '=', t3[j].x)
print("")
    
# x
print("x:")
print("j \ i", end="")
print("\tMachine2 Machine3 Machine4 Machine5")
for j, l in JL:
    # mark which product is printed now
    if len(str("Job" + str(j+1) + "-" + str(l+1))) < 7:
        print("Job" + str(j+1) + "-" + str(l+1), "\t", end="")
    else:
        print("Job" + str(j+1) + "-" + str(l+1), "", end="")
    for i in I:
        # print values of each kind of product
        if len(str(x3[i, j, l].x)) < 7:
            print(x3[i, j, l].x, "\t", end="")
        else:
            print(x3[i, j, l].x, "", end="")
    print("")    # use for change line
print("")

# z
print("z:")
print("j \ k", end="\t")
for j, l in JL:
    print("J" + str(j+1) + "-" + str(l+1), " ", end="")
print("")    # use for change line
for j, l in JL:
    # mark which product is printed now
    if len(str("Job" + str(j+1) + "-" + str(l+1))) < 7:
        print("Job" + str(j+1) + "-" + str(l+1), "\t", end="")
    else:
        print("Job" + str(j+1) + "-" + str(l+1), "", end="")
    for k, m in JL:
        # print values of each kind of product
        if (j, l) != (k, m):
            if len(str(z3[(j, l), (k, m)].x)) < 7:
                print(z3[(j, l), (k, m)].x, "  ", end="")
            else:
                print(z3[(j, l), (k, m)].x, "", end="")
        else:
            print("       ", end="")
    print("")    # use for change line
print("")

Result:
z* = 9.39999999999999

f[0,0] = 3.0 f[0,1] = 4.5 , Due time = 5 , t[0] = 0.0
f[1,0] = 3.0 f[1,1] = 5.0 , Due time = 5 , t[1] = 0.0
f[2,0] = 6.0 f[2,1] = 9.4 , Due time = 5 , t[2] = 1.0
f[3,0] = 3.0 f[3,1] = 5.0 , Due time = 5 , t[3] = 0.0
f[4,0] = 1.2 f[4,1] = 4.2 , Due time = 5 , t[4] = 0.0
f[5,0] = 0.0 f[5,1] = 5.0 , Due time = 5 , t[5] = 0.0
f[6,0] = 1.7 f[6,1] = 9.1 , Due time = 10 , t[6] = 0.0
f[7,0] = 0.0 f[7,1] = 8.9 , Due time = 10 , t[7] = 0.0
f[8,0] = 6.3 f[8,1] = 8.5 , Due time = 10 , t[8] = 0.0
f[9,0] = 7.6 f[9,1] = 8.9 , Due time = 10 , t[9] = 0.0

x:
j \ i	Machine2 Machine3 Machine4 Machine5
Job1-1 	-0.0 	0.0 	-0.0 	-0.0 	1.0 	
Job1-2 	1.0 	0.0 	-0.0 	-0.0 	-0.0 	
Job2-1 	-0.0 	0.0 	1.0 	-0.0 	-0.0 	
Job2-2 	-0.0 	-0.0 	-0.0 	-0.0 	1.0 	
Job3-1 	-0.0 	-0.0 	1.0 	-0.0 	-0.0 	
Job3-2 	-0.0 	0.0 	1.0 	-0.0 	-0.0 	
Job4-1 	1.0 	0.0 	-0.0 	-0.0 	-0.0 	
Job4-2 	-0.0 	-0.0 	-0.0 	1.0 	-0.0 	
Job5-1 	-0.0 	1.0 	-0.0 	-0.0 	-0.0 	
Job5-2 	-0.0 	0.0 	-0.0 	1.0 	-0.0 	
Job6-